In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# CIFAR-10 Dataset
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomAffine(0, translate=(0.2, 0.2), shear=20),
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

# Load Pretrained ResNet50
resnet_base = models.resnet50(pretrained=True)
for param in resnet_base.parameters():
    param.requires_grad = False

# Modify ResNet50 for CIFAR-10
resnet_base.fc = nn.Sequential(
    nn.Linear(resnet_base.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 10)
)

# Move model to GPU if available
model = resnet_base.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Using device: cuda
Files already downloaded and verified
Files already downloaded and verified
Epoch [1/50], Loss: 2.0125
Epoch [2/50], Loss: 1.8409
Epoch [3/50], Loss: 1.7946
Epoch [4/50], Loss: 1.7710
Epoch [5/50], Loss: 1.7514
Epoch [6/50], Loss: 1.7355
Epoch [7/50], Loss: 1.7224
Epoch [8/50], Loss: 1.7153
Epoch [9/50], Loss: 1.7191
Epoch [10/50], Loss: 1.7069
Epoch [11/50], Loss: 1.7049
Epoch [12/50], Loss: 1.6935
Epoch [13/50], Loss: 1.6920
Epoch [14/50], Loss: 1.6836
Epoch [15/50], Loss: 1.6821
Epoch [16/50], Loss: 1.6784
Epoch [17/50], Loss: 1.6825
Epoch [18/50], Loss: 1.6711
Epoch [19/50], Loss: 1.6701
Epoch [20/50], Loss: 1.6684
Epoch [21/50], Loss: 1.6684
Epoch [22/50], Loss: 1.6714
Epoch [23/50], Loss: 1.6578
Epoch [24/50], Loss: 1.6585
Epoch [25/50], Loss: 1.6643
Epoch [26/50], Loss: 1.6620
Epoch [27/50], Loss: 1.6580
Epoch [28/50], Loss: 1.6518
Epoch [29/50], Loss: 1.6520
Epoch [30/50], Loss: 1.6579
Epoch [31/50], Loss: 1.6535
Epoch [32/50], Loss: 1.6526
Epoch [33/50], Los